In [1]:
import json
# import yaml
import tensorflow as tf
from Exp_gain import *

In [2]:
def init_vec(dimension):
    return tf.Variable(tf.random_normal(dimension, stddev=0.01))

In [3]:
class DiverseRank(object):
    
    def __init__(self, lenepisode, nfeature, nhidden, lr):

        self.len_episode = lenepisode
        self.lr = lr
        self.nhidden = nhidden
        self.nfeature = nfeature

        global DD_learning_rate, DD_reward, DD_current_state, DD_query, DD_perceive_docs, DD_candidate_docs, DD_all_docs, DD_Ndoc

        global DD_query_state, DD_next_state, DD_score, DD_policy, DD_train_step, sess
        
        global cross_entropy,ep_split_train, DD_dynamic, score, policy

        with tf.name_scope('input'):
            DD_learning_rate = tf.placeholder(tf.float32)
            DD_Ndoc = tf.placeholder(tf.int64)
            DD_reward = tf.placeholder(tf.float32, None, name='reward')
            DD_current_state = tf.placeholder(tf.float32, [None, nhidden], name='current_state')
            DD_query = tf.placeholder(tf.float32, [None, nfeature], name='querys')
            DD_perceive_docs = tf.placeholder(tf.float32, [None, nfeature], name='docs')
            DD_candidate_docs = tf.placeholder(tf.float32, [None, nfeature], name='docs')
            DD_all_docs = tf.placeholder(tf.float32, [None, nfeature], name='candidate_docs')

        with tf.variable_scope('model_query') :
            W_q = init_vec([nfeature, nhidden])
            DD_query_state = tf.tanh(tf.matmul(DD_query, W_q))
            

        with tf.variable_scope('rnn1'):
            cell = tf.contrib.rnn.GRUCell(nhidden)
            ep_split = tf.split(DD_perceive_docs, 1, 0, 'split')
            _, DD_next_state = tf.contrib.rnn.static_rnn(cell, ep_split, initial_state=DD_current_state, dtype=tf.float32)

        with tf.variable_scope('policy'):
            W = init_vec([nfeature, nhidden])
            DD_score = tf.tanh(tf.matmul(DD_candidate_docs, tf.matmul(W, tf.transpose(DD_current_state))))
            DD_policy = tf.nn.softmax(tf.transpose(DD_score))

        
        with tf.variable_scope('rnn1', reuse=True) as scope:
            scope.reuse_variables()
            cell = tf.contrib.rnn.GRUCell(nhidden)
 
            ep_split_train = tf.split(DD_all_docs, 1000)               
            DD_dynamic, _ = tf.contrib.rnn.static_rnn(cell, ep_split_train, initial_state=DD_query_state, dtype=tf.float32)
            
            cross_entropy = 0
            for position in range(10):
                score = tf.tanh(tf.matmul(DD_all_docs[position:], tf.matmul(W, tf.reshape(DD_dynamic[position][1], [nhidden, 1]))))
                policy = tf.nn.softmax(tf.transpose(score))
                cross_entropy += DD_reward[position] * tf.nn.sparse_softmax_cross_entropy_with_logits(logits=policy, labels=[0])

        init = tf.global_variables_initializer()

        DD_train_step = tf.train.GradientDescentOptimizer(learning_rate=DD_learning_rate).minimize(cross_entropy)

        sess = tf.Session()
        sess.run(init)

    def gen_query_state(self, query):
        query_vec = np.asarray(query).reshape([1, self.nfeature])
        
        state = sess.run(DD_query_state, feed_dict={DD_query: query_vec})
        return state

    def gen_next_state(self, current_state, perceive_docs):
        current_state_vec = np.asarray(current_state).reshape([-1, self.nhidden])
        perceive_docs_vec = np.asarray(perceive_docs).reshape([-1, self.nfeature])
        
        state = sess.run(DD_next_state, feed_dict={DD_current_state: current_state_vec, DD_perceive_docs: perceive_docs_vec} )
        return state

    def gen_score(self, current_state, candidate_docs):
        current_state_vec = np.asarray(current_state).reshape([-1, self.nhidden])
        candidate_docs_vec = np.asarray(candidate_docs).reshape([-1, self.nfeature])
        
        score = sess.run(DD_score, feed_dict={DD_current_state: current_state_vec, DD_candidate_docs: candidate_docs_vec})
        return score

    def gen_policy(self, current_state, candidate_docs):
        current_state_vec = np.asarray(current_state).reshape([-1, self.nhidden])
        candidate_docs_vec = np.asarray(candidate_docs).reshape([-1, self.nfeature])
        
        policy = sess.run(DD_policy, feed_dict={DD_current_state: current_state_vec, DD_candidate_docs: candidate_docs_vec})
        return policy

    def gen_episode_softmax(self, queryvec, docvec, docs):
        current_state = self.gen_query_state(queryvec)

        ndoc = len(docvec)

        docd_id = docs[:]
        n_candidate = ndoc

        rank_list = []
        for ite in range(min(ndoc, self.len_episode)):
            policy = self.gen_policy(current_state, docvec)
            action = np.random.choice(n_candidate, 1, p=policy[0])[0]           

            current_state = self.gen_next_state(current_state, docvec[action])
            
            n_candidate = n_candidate - 1
            rank_list.append(docd_id[action])
            queryvec.append(docvec[action])
            del docd_id[action]
            del docvec[action]

        return rank_list, queryvec + docvec

    def gen_episode_greedy(self, queryvec, docvec, docs):

        current_state = self.gen_query_state(queryvec)

        Ndoc = len(docvec)
        Nite = min(Ndoc, self.len_episode)

        doc_id = docs[:]

        rank_list = []
        for ite in range(Nite):
            score = self.gen_score(current_state, docvec)
            action = np.argmax(score)

            current_state = self.gen_next_state(current_state, docvec[action])
            
            rank_list.append(doc_id[action])
            queryvec.append(docvec[action])
            del doc_id[action]
            del docvec[action]

        return rank_list, queryvec + docvec

    def test_model(self, querydocs, doc2vec, truth):
        meanreward = 0
        for query in querydocs.keys():
            docs = querydocs[query][:]

            ndoc = len(docs)
            if ndoc < 2:
                print '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!', query
                continue

            querydocsvec = []
            querydocsvec.append(doc2vec[query])
            docsvec = []
            for doc in docs:
                docsvec.append(doc2vec[doc])

            # gen episode
            ranklist, _ = self.gen_episode_greedy(querydocsvec, docsvec, docs)

            reward = alphaDCG_reward_per_query(ranklist, truth[query])
            # print reward[0],
            meanreward += reward[0]

        print '\n               ', meanreward, '\n'

    def train_model(self, querydocs, doc2vec, truth):
        querylist = querydocs.keys()
        train_data = {}
        for query in querylist:
            
            docs = querydocs[query][:]

            querydocsvec = []
            querydocsvec.append(doc2vec[query])
            docsvec = []
            for doc in docs:
                docsvec.append(doc2vec[doc])

            ranklist, querydocvec = self.gen_episode_softmax(querydocsvec, docsvec, docs)
            
            # print ranklist
            # print truth[query]
            reward = alphaDCG_reward_per_query(ranklist, truth[query])
            # print 'reward', reward[0]
            train_data[query] = {'querydocvec': querydocvec, 'reward': reward}
            
        print 'optimize'
        for query in querylist:
            querydocvec = train_data[query]['querydocvec']
            reward = train_data[query]['reward']
            self.optimize_model(querydocvec, reward)

    def optimize_model(self, querydocvec, reward):
        ndoc = len(querydocvec)
        # for i in range(len(reward)):
        #     if reward[i] == 0:
        #         continue
        ndoc = len(querydocvec)-1
        query_vec = np.asarray((querydocvec[0])).reshape([1, self.nfeature])
        doc_vec = np.asarray(querydocvec[1:ndoc+1]).reshape([-1, self.nfeature])
            
        qqq=sess.run(score, feed_dict={DD_query: query_vec, DD_all_docs: doc_vec, DD_Ndoc : ndoc, DD_reward:reward, DD_learning_rate: self.lr})
        print len(qqq)

    def main(self, nite, querydocs, doc2vec, query_document_topic):
        for ite in range(nite):
            self.train_model(querydocs, doc2vec, query_document_topic)

            if ite % 1 == 0:
                self.test_model(querydocs, doc2vec, query_document_topic)

In [4]:
Truth = json.load(open('/users/zengwei/data/DD/ZW_JSON/Truth.json'))
print 'load 1'
QueryDocs = json.load(open('/users/zengwei/data/DD/ZW_JSON/MiniData_QueryDocs.json'))
print 'load 3'
Doc2Vec = json.load(open('/users/zengwei/data/DD/ZW_JSON/MiniData_200_50ite_Doc2Vec.json'))
print 'load 4'

load 1
load 3


load 4


In [5]:
Nfeature=200
Nhidden = 10
Lenepisode = 50
lr=0.001

model = DiverseRank(Lenepisode, Nfeature, Nhidden, lr)

In [6]:
model.main(10000, QueryDocs, Doc2Vec, Truth)

optimize


InvalidArgumentError: slice index 1 of dimension 0 out of bounds.
	 [[Node: rnn1_1/strided_slice_1 = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=1, _device="/job:localhost/replica:0/task:0/cpu:0"](rnn1_1/rnn/rnn/gru_cell/add, rnn1_1/strided_slice_1/stack, rnn1_1/strided_slice_1/stack_1, rnn1_1/strided_slice_1/stack_2)]]

Caused by op u'rnn1_1/strided_slice_1', defined at:
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/users/zengwei/venv/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/users/zengwei/venv/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/users/zengwei/venv/lib/python2.7/site-packages/ipykernel/kernelapp.py",

In [7]:
type(Doc2Vec)

dict

In [8]:
help(tf.split)

Help on function split in module tensorflow.python.ops.array_ops:

split(value, num_or_size_splits, axis=0, num=None, name='split')
    Splits a tensor into sub tensors.
    
    If `num_or_size_splits` is an integer type, `num_split`, then splits `value`
    along dimension `axis` into `num_split` smaller tensors.
    Requires that `num_split` evenly divides `value.shape[axis]`.
    
    If `num_or_size_splits` is not an integer type, it is presumed to be a Tensor
    `size_splits`, then splits `value` into `len(size_splits)` pieces. The shape
    of the `i`-th piece has the same size as the `value` except along dimension
    `axis` where the size is `size_splits[i]`.
    
    For example:
    
    ```python
    # 'value' is a tensor with shape [5, 30]
    # Split 'value' into 3 tensors with sizes [4, 15, 11] along dimension 1
    split0, split1, split2 = tf.split(value, [4, 15, 11], 1)
    tf.shape(split0) ==> [5, 4]
    tf.shape(split1) ==> [5, 15]
    tf.shape(split2) ==> [5, 11]
 

In [2]:
import tensorflow as tf
import numpy as np

x = tf.placeholder(tf.int32, shape=[None, 2])
data = np.random.randint(10, size=(11,2))

parts = range(13)
out = tf.dynamic_partition(x, parts, 20)

sess = tf.Session()
print 'out tensors:\n', out
print
print 'input data:\n', data
print
print 'sess.run result:\n', sess.run(out, {x: data})

out tensors:
[<tf.Tensor 'DynamicPartition_1:0' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:1' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:2' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:3' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:4' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:5' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:6' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:7' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:8' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:9' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:10' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:11' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:12' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:13' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:14' shape=(?, 2) dtype=int32>, <tf.Tensor 'DynamicPartition_1:15' shape=(?, 2) dtype=int32>, <tf.

InvalidArgumentError: data.shape must start with partitions.shape, got data.shape = [11,2], partitions.shape = [13]
	 [[Node: DynamicPartition_1 = DynamicPartition[T=DT_INT32, num_partitions=20, _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_Placeholder_1_0_0, DynamicPartition_1/partitions)]]

Caused by op u'DynamicPartition_1', defined at:
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/users/zengwei/venv/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/users/zengwei/venv/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/users/zengwei/venv/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/users/zengwei/venv/lib/python2.7/